### scrape hub

In [89]:
#from urllib.request import urlopen, Request, urlretrieve
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import re

DRIVER_PATH = '/usr/sbin/chromedriver'
URLS = "./sorted.txt"
OUTPUT = "./output"
FAILED = "./failure.txt"

In [2]:
pwd

'/home/peat/hack/scwape/randodir'

In [90]:
options = Options()
options.headless = False
options.add_argument("--window-size=1920,1200")
profile = {"plugins.plugins_list": [{"enabled": False, "name": "Chrome PDF Viewer"}], # Disable Chrome's PDF Viewer
               "download.default_directory": OUTPUT ,
               "download.directory_upgrade": True,
               "plugins.always_open_pdf_externally": True,
               "download.extensions_to_open": "applications/pdf"}
options.add_experimental_option("prefs", profile)
driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
driver.implicitly_wait(0.5)
pdf= re.compile('^location\.href=\'(.*)\'$')
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
                      'AppleWebKit/537.11 (KHTML, like Gecko) '
                      'Chrome/23.0.1271.64 Safari/537.11',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
        'Accept-Encoding': 'none',
        'Accept-Language': 'en-US,en;q=0.8',
        'Connection': 'keep-alive'}

/tmp/ipykernel_25960/559492227.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)


In [64]:
filesizeregex = re.compile("  (\d*\.?\d+)(?=[KMGT])([KMGT])(?:i?B)?|B?$")

In [51]:
def scalar(bytescale):
    if bytescale[0] == 'b' or bytescale[0] == 'B':
        return 1
    elif bytescale[0] == 'k' or bytescale[0] == 'K':
        return 1000
    elif bytescale[0] == 'm' or bytescale[0] == 'M':
        return 1000000
    elif bytescale[0] == 'g' or bytescale[0] == 'G':
        return 1000000000
    elif bytescale[0] == 't' or bytescale[0] == 'T':
        return 1000000000000

In [97]:
start = 'https://www.kgay4all.com/'

def recurse(nuyurl):
        filesizes = 0
        dirlist = []
        filelist = []
        driver.switch_to.new_window('tab')
        t=driver.window_handles[-1]
        driver.switch_to.window(t)
        driver.get(nuyurl)
        WebDriverWait(driver, 5).until( lambda wd: driver.execute_script("return document.readyState") == 'complete', "Page taking too long to load")
        while(driver.title == 'HTML Redirect'):
                WebDriverWait(driver, 5).until( lambda wd: driver.execute_script("return document.readyState") == 'complete', "Page taking too long to load")
                #this is where you can add a proxy rotator after a certain amount of non-loading
        elems = driver.find_elements(By.XPATH, "//a[@href]")
        #print(driver.page_source)
        for filesize in filesizeregex.findall(driver.page_source):
                if(filesize[0] != ''):
                        filesizes = filesizes + int(filesize[0]) * scalar(filesize[1])
        for elem in elems:
                href = elem.get_attribute("href")
                if href != nuyurl and href not in nuyurl:
                        if href.endswith('/'):
                                (temp_dir, temp_file, temp_filesize) = recurse(href)
                                dirlist = dirlist + [href]
                                dirlist = dirlist + temp_dir
                                filelist = filelist + temp_file
                                filesizes = filesizes + temp_filesize
                                driver.switch_to.window(t)
                        else:
                                filelist = filelist + [href]
                #print(elem.get_attribute("href"))
        driver.close()
        return (dirlist, filelist, filesizes)

In [98]:
recursed = recurse(start)

In [54]:
print(recursed)

([], [], 0)


## OLD NOT USED